In [1]:
# EDA e Visualização de Dados
import pandas as pd
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# ML
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Otimização HP
import optuna

### Carregar os Dados

In [2]:
# Carga de Dados
df_clientes = pd.read_csv("./datasets/dataset_clientes_pj.csv")

In [3]:
# Visualizar a estrutura
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [4]:
# Visualizar os primeiros registros
df_clientes.head(5)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8


### EDA

In [5]:
# Distribuição da variável inovação
percentual_inovacao = df_clientes.value_counts('inovacao') / len(df_clientes) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [6]:
# Teste ANOVA (Análise de Variância)
# Verificar se há variações significativas na média de faturamento mensal para diferentes níveis de inovação
# Suposições / Pressupostos:
# - Observações independentes
# - Variável dependente é contínua
# - Segue uma distribuição normal
# - Homogeneidade das variâncias
# - Amostras sejam de tamanhos iguais

In [7]:
# Checar se as variâncias (faturamento) entre os grupos (inovação) são homogêneas
# Aplicar Teste de Bartlett
# H0 - Variâncias são iguais
# H1 - Variâncias não são iguais
# Se p-value for maior que 0,05, então aceitamos a hipótese nula (H0)
# Se p-value for menor que 0,05, então rejeitamos a hipótese nula

from scipy.stats import bartlett

# Separando os dados de faturamento em grupos com base na coluna "inovação"
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()]

# Executar o teste de Bartlett
bartlett_test_statistic, bartlett_p_value = bartlett(*dados_agrupados)

# Exibindo os resultados
print(f"Estatística do teste de Bartlett: {bartlett_test_statistic}")
print(f"P-Value do teste de Bartlett: {bartlett_p_value}")

Estatística do teste de Bartlett: 10.901203117231173
P-Value do teste de Bartlett: 0.28254182954905804


In [8]:
# Executar o Teste de Shapiro-Wilk
# Verificar se os dados seguem uma distribuição normal
# H0 - Segue uma distribuição normal
# H1 - Não segue uma distribuição normal
# Se p-value for maior que 0,05, então aceitamos a hipótese nula (H0)
# Se p-value for menor que 0,05, então rejeitamos a hipótese nula

from scipy.stats import shapiro

# Executar o teste
shapiro_test_statistic, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

# Exibindo os resultados
print(f"Estatística do teste de SW: {shapiro_test_statistic}")
print(f"P-Value do teste de SW: {shapiro_p_value}")

Estatística do teste de SW: 0.9959857602472711
P-Value do teste de SW: 0.23513451034389005


In [9]:
# Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes
# H0 - Não há diferenças significativas entre as médias dos grupos
# H1 - Há pelo menos uma diferença significativa entres as médias dos grupos
# Se p-value for maior que 0,05, então aceitamos a hipótese nula (H0)
# Se p-value for menor que 0,05, então rejeitamos a hipótese nula

from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clientes)

# Exibindo os resultados
print(f"Estatística do teste de ANOVA Welch: {aov.loc[0, 'F']}")
print(f"P-Value do teste de ANOVA Welch: {aov.loc[0, 'p-unc']}")

Estatística do teste de ANOVA Welch: 1.1269836194061693
P-Value do teste de ANOVA Welch: 0.3452621127391258


### Treinar o algoritmo K-Means

In [10]:
# Selecionar as colunas para clusterização
X = df_clientes.copy()

# Separando variáveis numéricas, categóricas e ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

# Aplicar Transformações por tipo
numeric_tranformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_tranformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
  transformers=[
    ('num', numeric_tranformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('ord', ordinal_tranformer, ordinal_features)
  ]
)

# Transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [11]:
# Optuna para Otimização de Hiperparâmetros
def kmeans_objective(trial):

  # Definir os hiperparâmetros a serem ajustados
  n_clusters = trial.suggest_int('n_clusters', 3, 10)
  distance_metric = trial.suggest_categorical('distance_metric', ['euclidean', 'minkowski'])

  # Criar o Modelo
  modelo_kmeans = KMeans(n_clusters=n_clusters, random_state=51)

  # Treinar o modelo
  modelo_kmeans.fit(X_transformed)

  # Calculando o Silhouette Score
  distances = pairwise_distances(X_transformed, metric=distance_metric)
  silhouette_avg = silhouette_score(distances, modelo_kmeans.labels_)

  return silhouette_avg

In [12]:
# Criar um estudo do Optuna
search_space = {'n_clusters': [3, 4, 5, 6, 7, 8, 9 , 10], 'distance_metric': ['euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

# Rodar o estudo
estudo_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2024-11-22 11:23:06,166] A new study created in memory with name: no-name-4429e541-46cf-4bd7-ad36-92bac80f4305


[I 2024-11-22 11:23:06,413] Trial 0 finished with value: 0.38478794965029234 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.38478794965029234.
[I 2024-11-22 11:23:06,591] Trial 1 finished with value: 0.11230048381589083 and parameters: {'n_clusters': 9, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.38478794965029234.
[I 2024-11-22 11:23:06,761] Trial 2 finished with value: 0.4445458290999088 and parameters: {'n_clusters': 3, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4445458290999088.
[I 2024-11-22 11:23:06,889] Trial 3 finished with value: 0.38478794965029234 and parameters: {'n_clusters': 4, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4445458290999088.
[I 2024-11-22 11:23:06,981] Trial 4 finished with value: 0.14731572416665722 and parameters: {'n_clusters': 8, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4445458290999088.
[I 2024-11-22 11:23:07,108] Trial 5 fin

In [13]:
# Melhor configuração encontrada pelo Optuna
best_params = estudo_kmeans.best_params

# Instanciando o modelo K-Means com melhores parâmetros
best_kmeans = KMeans(n_clusters=best_params["n_clusters"], random_state=51)
best_kmeans.fit(X_transformed)

# Calculando o Silhouette Score
distances = pairwise_distances(X_transformed, metric=best_params["distance_metric"])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

print(f'k (Numero de Clusters): {best_params["n_clusters"]}')
print(f'Métrica de Distância Selecionada: {best_params["distance_metric"]}')
print(f'Silhouette Score: {best_silhouette}')

k (Numero de Clusters): 3
Métrica de Distância Selecionada: minkowski
Silhouette Score: 0.4445458290999088


In [14]:
# Criar coluna com cluster escolhido
df_clientes['cluster'] = best_kmeans.labels_

In [15]:
# Visualizar os primeiros registros
df_clientes.head(5)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1


### Visualizar Resultados

In [16]:
# Cruzar idade e faturamento, apresentando os clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='cluster')

In [17]:
# Cruzar inovação e faturamento, apresentando os clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='cluster')

In [18]:
# Cruzar número de funcionários e faturamento, apresentando os clusters
px.scatter(df_clientes, x='numero_de_funcionarios', y='faturamento_mensal', color='cluster')

### Salvar o Modelo e o Pipeline de Transformação

In [19]:
import joblib

# Salvar o modelo
joblib.dump(best_kmeans, 'modelo_clusterizacao_clientes.pkl')

# Salvar o Pipeline
joblib.dump(preprocessor, 'pipeline_clusterizacao_clientes.pkl')

['pipeline_clusterizacao_clientes.pkl']

### Aplicação Batch no Gradio

In [20]:
import gradio as gr

modelo = joblib.load('./modelo_clusterizacao_clientes.pkl')
preprocessor = joblib.load('./pipeline_clusterizacao_clientes.pkl')

def clustering(arquivo):
  # Carregar o CSV em um Dataframe
  df_empresas = pd.read_csv(arquivo.name)

  # Transformar os dados do DF para o formato que o KMeans precisa
  X_transformed = preprocessor.fit_transform(df_empresas)

  # Treinar modelo
  modelo.fit(X_transformed)

  # Criar a coluna cluster no DF
  df_empresas['cluster'] = modelo.labels_
  df_empresas.to_csv('./clusters.csv', index=False)

  return './clusters.csv'

In [21]:
# Criar a interface
app = gr.Interface(
  clustering,
  gr.File(file_types=[".csv"]),
  "file"
)

# Rodar a aplicação
app.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
